[깃허브 출처 링크 : KorEDA](https://github.com/catSirup/KorEDA/blob/master/eda.py)

## import

In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/2024 학교/졸작 준비/인공지능/최종 코드')

In [ ]:
import pandas as pd

In [ ]:
import random
import pickle
import re

wordnet = {}
with open("./wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)

In [ ]:
# 한글만 남기고 나머지는 삭제
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText



########################################################################
# Synonym replacement
# Replace n words in the sentence with synonyms from wordnet
########################################################################
def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################
def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)

	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return

	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:129: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:129: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-5-e71f13b7e322>:129: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


In [ ]:
# 증강할 문장
sentence = "야. 죽기 싫으면 내 말 들어라. 가진 거 다 안 내놓으면 내 손에 죽는 거야."

# EDA 함수 호출
augmented_sentences = EDA(sentence, alpha_sr=0.2, alpha_ri=0.1, alpha_rs=0.3, p_rd=0.3, num_aug=3)
# 결과 출력
for idx, aug_sentence in enumerate(augmented_sentences):
    print(f"증강된 문장 {idx + 1}: {aug_sentence}")

증강된 문장 1: 야. 죽기 싫으면 내 말 들어라. 가진 거 다 안 내놓으면 내 손에 죽는 거야.
증강된 문장 2: 야. 죽기 말 싫으면 내 말 들어라. 가진 거 다 안 내놓으면 내 손에 죽는 거야.
증강된 문장 3: 야. 죽기 내놓으면 싫으면 내 들어라. 손에 거 다 안 내 말 가진 죽는 거야.
증강된 문장 4: 야. 죽기 싫으면 내 말 들어라. 가진 거 다 안 내놓으면 내 손에 죽는 거야.


In [ ]:
# 증강할 문장
sentence = "제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요"

# EDA 함수 호출
augmented_sentences = EDA(sentence, alpha_sr=0.2, alpha_ri=0.1, alpha_rs=0.3, p_rd=0.3, num_aug=3)

# 결과 출력
for idx, aug_sentence in enumerate(augmented_sentences):
    print(f"증강된 문장 {idx + 1}: {aug_sentence}")

증강된 문장 1: 제가 점점 느낀지는 오래됐는데 우울감을 있다고 개선되고 느껴요
증강된 문장 2: 우울감을 느낀지는 오래됐는데 점점 개선되고 느껴요
증강된 문장 3: 제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요
증강된 문장 4: 제가 우울감을 느낀지는 오래됐는데 점점 개선되고 있다고 느껴요


## 가지고 있는 데이터로 증강 실행

In [ ]:
emer_data = pd.read_csv('/content/drive/MyDrive/2024 학교/졸작 준비/인공지능/data 최종/위험데이터_전처리완료_2.csv')

In [ ]:
emer_data

,note
0,내 몸에 손대지 마.
1,내 몸에 손대지 마세요.
2,제 몸에 손대지 마세요.
3,손대지 마.
4,내 몸 만지지 마.
...,...
2470,여기 삼호동 와와 공원인데요. 어르신께서 넘어져서 다치셨어요.
2471,어르신이 식사하러 오셨는데 집을 잘 모른다고 하십니다.
2472,좀 도와주시면 감사하겠습니다.
2473,어르신께서 길을 잃으셔서 연락 드립니다.


In [ ]:
# 각 문장에 대해 4개의 증강 문장 생성 및 추가
def augment_dataframe(df, column_name):
    for i in range(1, 5):
        df[f'augmented_sentence_{i}'] = df[column_name].apply(lambda x: EDA(x, num_aug=4)[i-1])
    return df

# 데이터프레임에 증강 문장 추가
emer_data = augment_dataframe(emer_data, 'note')

# 결과 출력
print(emer_data)

                                    note                augmented_sentence_1  \
0                            내 몸에 손대지 마.                         내 몸에 손대지 마.   
1                          내 몸에 손대지 마세요.                       내 몸에 손대지 마세요.   
2                          제 몸에 손대지 마세요.                       손대지 몸에 제 마세요.   
3                                 손대지 마.                              손대지 마.   
4                             내 몸 만지지 마.                          마. 몸 만지지 내   
...                                  ...                                 ...   
2470  여기 삼호동 와와 공원인데요. 어르신께서 넘어져서 다치셨어요.  여기 삼호동 와와 공원인데요. 어르신께서 넘어져서 다치셨어요.   
2471      어르신이 식사하러 오셨는데 집을 잘 모른다고 하십니다.      어르신이 식사하러 오셨는데 집을 잘 모른다고 하십니다.   
2472                    좀 도와주시면 감사하겠습니다.                    좀 도와주시면 감사하겠습니다.   
2473              어르신께서 길을 잃으셔서 연락 드립니다.              어르신께서 길을 잃으셔서 연락 드립니다.   
2474          배회 인식표를 하고 계시는데 확인 부탁드립니다.           배 인식표를 하고 계시는데 확인 부탁드립니다.   

                    augmented_sentence_

In [ ]:
emer_data.to_csv('/content/drive/MyDrive/2024 학교/졸작 준비/인공지능/data 최종/08.18_위험데이터_EDA증강.csv', index = None)

- SR: Synonym Replacement, 특정 단어를 유의어로 교체
- RI: Random Insertion, 임의의 단어를 삽입
- RS: Random Swap, 문장 내 임의의 두 단어의 위치를 바꿈
- RD: Random Deletion: 임의의 단어를 삭제

**각 인자 설명**
- alpha_sr, alpha_ri, alpha_rs: 각 기법에 의해 변형될 단어의 비율을 설정합니다. 예를 들어 alpha_sr=0.1이면, 문장의 단어 중 10%가 동의어로 치환됩니다.
- p_rd: 랜덤 삭제 시 단어가 삭제될 확률입니다.
- num_aug: 생성할 증강 문장의 개수를 설정합니다.

**저자의 설명**
- WordNet만을 단순히 바꿔서 결괏값을 내기 때문에 의미가 변형되어버리는 경우가 생깁니다. 특히 SR과 RI를 사용할 때 많이 발생하는데 제가 잘못한 건 아닌 것 같아요 를 제가 잘못한 총 아닌 것 같아요 (건 -> 총) 으로 바뀌기도 한다. 본 논문에서는 이렇게 바꿔도 꽤나 원문 데이터의 성질을 따라간다고 하지만.. 한국어의 특성상 완전히 따라가기에는 쉽지 않은 것 같다.

- 안전하게 데이터 증강을 하고 싶다면 RD, RS만을 사용하고, 데이터가 많이 필요하다싶으면 SR과 RI까지 사용하고 인간지능으로 데이터를 걸러내는 작업이 필요할 것이다.